In [147]:
import requests
import pandas as pd
import json
from tqdm import tqdm 
from json.decoder import JSONDecodeError

In [186]:
# importando .csv com dados de nome e região de cada estado
info_ufs = pd.read_csv('dados_estados.csv')
info_ufs.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


### Dados totais de nascimentos por estado

In [49]:
# para evitar erro 403 (sem autorização)
session = requests.Session()
session.headers.update({'User-Agent': 'Custom user agent'})

# raspando dados de registros totais por uf
resposta_estados = session.get('https://transparencia.registrocivil.org.br/api/record/birth')
resposta_estados = resposta_estados.json()

In [51]:
resposta_estados['data']

[{'name': 'AC', 'total': 13247},
 {'name': 'AL', 'total': 4121},
 {'name': 'AP', 'total': 23974},
 {'name': 'AM', 'total': 2971},
 {'name': 'BA', 'total': 13169},
 {'name': 'CE', 'total': 3738},
 {'name': 'DF', 'total': 13598},
 {'name': 'ES', 'total': 20483},
 {'name': 'GO', 'total': 9999},
 {'name': 'MA', 'total': 438},
 {'name': 'MT', 'total': 11},
 {'name': 'MS', 'total': 1530},
 {'name': 'MG', 'total': 30487},
 {'name': 'PA', 'total': 444},
 {'name': 'PB', 'total': 715},
 {'name': 'PR', 'total': 5367},
 {'name': 'PE', 'total': 88882},
 {'name': 'PI', 'total': 274},
 {'name': 'RJ', 'total': 34150},
 {'name': 'RN', 'total': 434},
 {'name': 'RS', 'total': 408938},
 {'name': 'SC', 'total': 112985},
 {'name': 'SP', 'total': 1285925},
 {'name': 'SE', 'total': 1788},
 {'name': 'TO', 'total': 6}]

### rapando dados do ranking dos 50 principais nomes por estado

In [184]:
estados = []
df_estados = pd.DataFrame()
for uf in info_ufs.values: #resposta_estados['data']
    
    uf_nome = uf[1]
    resposta = session.get(f'https://transparencia.registrocivil.org.br/api/record/all-name?start_date=2021-01-01&end_date=2021-12-31&translate=1&state={uf_nome}')
    print(f"{uf[1]}: {resposta.status_code}")
    resposta = resposta.json()
    df_resposta = pd.DataFrame(resposta['data'])
    df_resposta['uf'] = uf_nome
    df_resposta['regiao'] = uf[5]
    df_resposta['ano'] = 2021
    df_resposta.index = df_resposta.index + 1
    
    df_estados = df_estados.append(df_resposta)
    
    print('---')

RO: 200
---
AC: 200
---
AM: 200
---
RR: 200
---
PA: 200
---
AP: 200
---
TO: 200
---
MA: 200
---
PI: 200
---
CE: 200
---
RN: 200
---
PB: 200
---
PE: 200
---
AL: 200
---
SE: 200
---
BA: 200
---
MG: 200
---
ES: 200
---
RJ: 200
---
SP: 200
---
PR: 200
---
SC: 200
---
RS: 200
---
MS: 200
---
MT: 200
---
GO: 200
---
DF: 200
---


In [185]:
df_estados.head()

,total,name,uf,regiao,ano
1,288,MIGUEL,RO,Norte,2021
2,268,HEITOR,RO,Norte,2021
3,266,ARTHUR,RO,Norte,2021
4,186,LAURA,RO,Norte,2021
5,183,GAEL,RO,Norte,2021


In [106]:
df.to_csv('nomes_mais_registrados_br_2021.csv')

### Dados por município

In [111]:
resposta_municipios = session.get('https://transparencia.registrocivil.org.br/api/cities')
resposta_municipios = resposta_municipios.json()

In [183]:
resposta_municipios['cities'][0:11]

[{'uf': 'SP', 'name': 'São Paulo', 'id': 1},
 {'uf': 'SP', 'name': 'São Bernardo do Campo', 'id': 2},
 {'uf': 'SP', 'name': 'Sumare', 'id': 3},
 {'uf': 'SP', 'name': 'São Caetano do Sul', 'id': 4},
 {'uf': 'SP', 'name': 'Santo Andre', 'id': 5},
 {'uf': 'SP', 'name': 'Barueri', 'id': 6},
 {'uf': 'SP', 'name': 'Ribeirão Pires', 'id': 7},
 {'uf': 'SP', 'name': 'Campinas', 'id': 8},
 {'uf': 'SP', 'name': 'Paulinia', 'id': 9},
 {'uf': 'SP', 'name': 'Salto', 'id': 10},
 {'uf': 'SP', 'name': 'Cotia', 'id': 11}]

#### Tranformando nome dos municípios em no formato URL encode

In [125]:
# testanto biblioteca
import urllib.parse
query = 'Jequiá da Praia'
urllib.parse.quote(query)


'Jequi%C3%A1%20da%20Praia'

### rapando dados dos municípios brasileiros

In [160]:

df_municipios = pd.DataFrame()

for item in tqdm(resposta_municipios['cities']):
    try:
        municipio_encode = urllib.parse.quote(item['name'])        
        uf_busca = item['uf']     
        resposta = session.get(f'https://transparencia.registrocivil.org.br/api/record/all-name?start_date=2021-01-01&end_date=2021-12-31&translate=1&state={uf_busca}&city={municipio_encode}')

        resposta = resposta.json()
        df_resposta = pd.DataFrame(resposta['data'])
        df_resposta['uf'] = uf_busca
        df_resposta['municipio'] = item['name']
        df_resposta['ano'] = 2021
        df_resposta.index = df_resposta.index + 1
        #display(df_resposta)
        df_municipios = df_municipios.append(df_resposta)

    except json.decoder.JSONDecodeError:
        print(item['name']) 
        print(item['uf'])
        print(resposta.status_code)
        print("String could not be converted to JSON")
        continue  
         



100%|████████████████████████████████████████████████████████████████████████████| 5611/5611 [1:58:30<00:00,  1.27s/it]


In [166]:
#total de municípios com dados coletados
len(df_municipios.groupby('municipio').sum())

4835

In [169]:
df_municipios.to_csv('nomes_mais_registrados_municipios_2021.csv')